# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [1]:
# import libraries
import numpy as np
### Spark
import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.functions import avg, col, concat, count, desc, explode, lit, min, max, split, stddev, udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum
from pyspark.ml import Pipeline
from pyspark.ml.feature import RegexTokenizer, CountVectorizer, VectorAssembler, Normalizer, StandardScaler, IDF, StringIndexer
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.clustering import KMeans
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, RegressionEvaluator

In [2]:
# create a Spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [3]:
# load 'mini_sparkify_event_data.json'
sparkify = spark.read.json('mini_sparkify_event_data.json')
sparkify.persist()
sparkify.createOrReplaceTempView('sparkify')
sparkify.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [4]:
# 2107 users who "churned"
churned_users = spark.sql("select distinct userId from sparkify \
                            WHERE (page = 'Cancellation Confirmation') or (page = 'Downgrade') \
                            order by userId")
churned_users.show()
print(churned_users.count())

+------+
|userId|
+------+
|    10|
|   100|
|100001|
|100002|
|100003|
|100004|
|100005|
|100006|
|100007|
|100008|
|100009|
|100011|
|100012|
|100013|
|100014|
|100015|
|100016|
|100017|
|100018|
|100019|
+------+
only showing top 20 rows

171


In [5]:
# songs list of users who churned = 217405 total
table = spark.sql(''' ((select *, 1 as churn 
            from sparkify 
              where page = 'NextSong' and userId IN (
                select distinct userId from sparkify 
                WHERE (page = 'Cancellation Confirmation') or (page = 'Downgrade') 
                order by userId) 
                ) )
            union ( 
            (select *, 0 as churn 
            from sparkify where page = 'NextSong' and userId not IN ( 
            select distinct userId from sparkify 
            WHERE (page = 'Cancellation Confirmation') or (page = 'Downgrade') 
            order by userId) 
             )  )
             ''')

table.createOrReplaceTempView('table')
df = spark.sql("select * from table where churn = 1").show()
df

+--------------------+---------+---------+------+-------------+----------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+-----+
|              artist|     auth|firstName|gender|itemInSession|  lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|churn|
+--------------------+---------+---------+------+-------------+----------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+-----+
|    Dario Marianelli|Logged In|     Alex|     M|            4|     Hogan|146.99057| free|Denver-Aurora-Lak...|   PUT|NextSong|1535066380000|      100|        Cello Lesson|   200|1538359289000|Mozilla/5.0 (Wind...|   101|    1|
|             BlÃÂ¸f|Logged In|    Grant|     M|           78|    Flores|202.84036| pai

In [6]:
### self-input
'''
We can see that most people Downgrade their service rather than cancel the service entirely.
Churn: 21 (went free) / 2086 (downgraded the subscription) [total 2107]
In total 52 people cancelled
Cancellation/downgrade is pretty distributed between Males and Females.
'''

'\nWe can see that most people Downgrade their service rather than cancel the service entirely.\nChurn: 21 (went free) / 2086 (downgraded the subscription) [total 2107]\nIn total 52 people cancelled\nCancellation/downgrade is pretty distributed between Males and Females.\n'

In [7]:
# SELECT CASE WHEN 1 > 0 THEN 1 WHEN 2 > 0 THEN 2.0 ELSE 1.2 END;
is_home = spark.sql("SELECT *, CASE WHEN page = 'NextSong' THEN 1 ELSE 0 END AS is_next_song FROM table \
             WHERE page = 'NextSong'") #

# keep the results in a new view
is_home.createOrReplaceTempView("is_next_song_table")

# find the cumulative sum over the is_home column
cumulative_sum = spark.sql("SELECT *, SUM(is_next_song) OVER \
    (PARTITION BY userID ORDER BY ts ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS period \
    FROM is_next_song_table \
    ORDER BY userId ASC, ts DESC")

# keep the results in a view
cumulative_sum.createOrReplaceTempView("song_count_table")

# find the average count for NextSong
#spark.sql("SELECT AVG(count_results) FROM \
#          (SELECT COUNT(*) AS count_results FROM song_count_table \
#GROUP BY userID, period, page HAVING page = 'NextSong') AS counts").show()

dataframe = spark.sql('''SELECT userId, first(itemInSession) as itemInSession, first(churn) as churn, COUNT(*) AS song_count FROM song_count_table \
            GROUP BY userId
            ORder By userId ASC
            ''')
dataframe.show()
print(dataframe.count())

+------+-------------+-----+----------+
|userId|itemInSession|churn|song_count|
+------+-------------+-----+----------+
|    10|           97|    1|       673|
|   100|           91|    1|      2682|
|100001|           20|    1|       133|
|100002|            0|    1|       195|
|100003|           78|    1|        51|
|100004|          121|    1|       942|
|100005|           69|    1|       154|
|100006|           42|    1|        26|
|100007|          198|    1|       423|
|100008|           42|    1|       772|
|100009|           58|    1|       518|
|100010|           33|    0|       275|
|100011|           19|    1|        11|
|100012|           71|    1|       476|
|100013|           48|    1|      1131|
|100014|           70|    1|       257|
|100015|          112|    1|       800|
|100016|           48|    1|       530|
|100017|           71|    1|        52|
|100018|           49|    1|      1002|
+------+-------------+-----+----------+
only showing top 20 rows

225


# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [8]:
assembler = VectorAssembler(inputCols=["itemInSession", "song_count"], outputCol="featureVec")
df_assembled = assembler.transform(dataframe)

In [9]:
# Features: An array of data points of features to be used for predicting (the label).
# Label: the output for each data point.
# Required `label` to be Int: It already is an Int
data = df_assembled.select(col("churn").alias("label"), col("featureVec").alias("features"))
data.head()

Row(label=1, features=DenseVector([97.0, 673.0]))

In [10]:
data.show()

+-----+-------------+
|label|     features|
+-----+-------------+
|    1| [97.0,673.0]|
|    1|[91.0,2682.0]|
|    1| [20.0,133.0]|
|    1|  [0.0,195.0]|
|    1|  [78.0,51.0]|
|    1|[121.0,942.0]|
|    1| [69.0,154.0]|
|    1|  [42.0,26.0]|
|    1|[198.0,423.0]|
|    1| [42.0,772.0]|
|    1| [58.0,518.0]|
|    0| [33.0,275.0]|
|    1|  [19.0,11.0]|
|    1| [71.0,476.0]|
|    1|[48.0,1131.0]|
|    1| [70.0,257.0]|
|    1|[112.0,800.0]|
|    1| [48.0,530.0]|
|    1|  [71.0,52.0]|
|    1|[49.0,1002.0]|
+-----+-------------+
only showing top 20 rows



# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [11]:
train, test = data.randomSplit([0.7, 0.3], seed=42)

In [12]:
##### ##### Linear Regression ##### #####

In [13]:
# Linear Regression
linreg = LinearRegression(maxIter=10, regParam=0.0, fitIntercept=False, solver="normal")

model = linreg.fit(train)
print(model.summary.residuals.show())
print(model.summary.rootMeanSquaredError)
print(model.summary.r2)
print(model.coefficients)

+--------------------+
|           residuals|
+--------------------+
|-0.02423586435434788|
|   0.917125058533248|
|  0.9392372093638337|
|  0.8225249445901028|
|  0.8090739479675788|
|  0.6724353052457737|
|   0.957212333298287|
|  0.7045942821830341|
|  0.5481399690006165|
|   0.774907150238884|
|  0.5862653119262324|
|  0.8522425817599869|
|  0.7587955173065957|
|  0.9289257273708934|
|  0.7520213281848384|
| 0.14588313827601396|
|  0.6552909489207277|
| 0.08038464593503336|
|  0.7074904965779567|
|-0.11187145929842292|
+--------------------+
only showing top 20 rows

None
0.532588626590456
0.6256741485825015
[0.00207158011062,0.000311604054544]


In [14]:
results = model.transform(test)

In [15]:
pred_res = results.filter(results.label == results.prediction).count()
total = results.count()
print(pred_res)
print(total)
print(pred_res/total) # 0.0

0
64
0.0


In [16]:
##### ##### Logistic Regression ##### #####

In [17]:
# Logistic Regression
logreg = LogisticRegression(maxIter=10, regParam=0.0)

model = logreg.fit(train) # used to be data
# print(model.summary.residuals.show())
print(model.summary.accuracy) # 0.8385093167701864
print(model.coefficients)

0.8385093167701864
[0.00229161062917,0.00496219529871]


In [18]:
results = model.transform(test) # used to be data

In [19]:
pred_res = results.filter(results.label == results.prediction).count()
total = results.count()
print(pred_res) # 58
print(total) # 64
print(pred_res/total) # 0.90625

58
64
0.90625


# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.

In [20]:
##### ##### Linear Regression ##### #####

In [21]:
'''
No need to make a cross validation where the model is not performing
'''

'\nNo need to make a cross validation where the model is not performing\n'

In [22]:
##### ##### Logistic Regression ##### #####

In [23]:
# Tuning
paramGrid = ParamGridBuilder() \
     .addGrid(logreg.maxIter,[5, 10, 15, 25]) \
     .addGrid(logreg.regParam,[0.0, 0.5, 1.0]) \
     .build()
    
crossval = CrossValidator(estimator=logreg,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(),
                          numFolds=3)

cvmodel = crossval.fit(train)
cvmodel.avgMetrics

[0.4295431762135385,
 0.49065256832765525,
 0.49065256832765525,
 0.46736471985619393,
 0.49065256832765525,
 0.49065256832765525,
 0.46736471985619393,
 0.49065256832765525,
 0.49065256832765525,
 0.46736471985619393,
 0.49065256832765525,
 0.49065256832765525]

In [24]:
results = cvmodel.transform(test)

In [25]:
pred_res = results.filter(results.label == results.prediction).count()
total = results.count()
print(pred_res)
print(total)
print(pred_res/total)

54
64
0.84375


In [26]:
print("END")

END
